In [2]:
import re
import sys
import nltk
import spacy
import numpy as np
import pandas as pd

import multiprocessing
from functools import partial
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

In [4]:
#!{sys.executable} -m pip install contractions
import contractions

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
rawdf = pd.read_csv('merged_ls_200923.csv', engine="python", on_bad_lines="skip")
rawdf.dropna(inplace=True)

In [7]:
print(rawdf.shape[0])

21210


In [8]:
rawdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21210 entries, 0 to 23667
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    21210 non-null  object
 1   label   21210 non-null  object
dtypes: object(2)
memory usage: 497.1+ KB


In [9]:
labels = ['AVI', 'MIS', 'OTH', 'WIN', 'WEA',
           'COC', 'PRP', 'MEC', 'ELC', 'LAG',
           'FLC', 'FFC', 'ECS', 'FLU']

def filter_by_label(df):
  filtered_df = rawdf[rawdf['label'].apply(lambda x: isinstance(x, str) and x in labels)]
  return filtered_df

df = filter_by_label(rawdf)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20867 entries, 0 to 23667
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    20867 non-null  object
 1   label   20867 non-null  object
dtypes: object(2)
memory usage: 489.1+ KB


In [11]:
class_counts = df.groupby('label').size().reset_index(name='count')
print(class_counts)

   label  count
0    AVI   7775
1    COC   1738
2    ECS     48
3    ELC     60
4    FFC    132
5    FLC    528
6    FLU     26
7    LAG    140
8    MEC    129
9    MIS   4131
10   OTH   4044
11   PRP    453
12   WEA   1573
13   WIN     90


In [12]:
# nlp = spacy.load("en_core_web_sm")
# # stemmer = PorterStemmer()
# # lemmatizer = WordNetLemmatizer()
# # stop_words = set(stopwords.words('english'))


# def expand_contractions(text):
#     contractions_pattern = re.compile(r"(?i)(\w+)(n't|'ve|'ll|'d|'re|'s|'m)")
#     def replace(match):
#         contraction, suffix = match.groups()
#         if suffix == "n't":
#             return contraction + " not"
#         elif suffix in ["'ve", "'ll", "'d"]:
#             return contraction + " " + suffix
#         elif suffix == "'re":
#             return contraction + " are"
#         elif suffix == "'s":
#             return contraction + " 's"
#         elif suffix == "'m":
#             return contraction + " am"

#     return contractions_pattern.sub(replace, text)


# def remove_special_characters(text):
#     pattern = re.compile(r"[#%@\<>{}()=\[\]*\-\\|_½]")
#     return pattern.sub("", text.lower())


# def clean_text(text):

#     text = remove_special_characters(text)
#     #text = expand_contractions(text)
#     text = contractions.fix(text)

#     doc = nlp(text)

#     lemmatized_tokens = [token.lemma_ for token in doc]
#     lemmatized_tokens = [token.lower() for token in lemmatized_tokens]
#     lemmatized_tokens = [token for token in lemmatized_tokens if not nlp.vocab[token].is_stop]

#     return lemmatized_tokens

# processed_texts = list(nlp.pipe(df['text'], batch_size=500, n_process=-1))

# preprocessed_texts = [clean_text(text) for text in processed_texts]

# df['text'] = preprocessed_texts

In [13]:
def expand_contractions(text):
    contractions = {
        "n't": "not",
        "'ve": "have",
        "'ll": "will",
        "'d": "would",
        "'re": "are",
        "'s": "is",
        "'m": "am"
    }
    pattern = re.compile(r"\b(?:" + "|".join(contractions.keys()) + r")\b")
    return pattern.sub(lambda match: contractions[match.group(0)], text)

# Function to remove special characters
def remove_special_characters(text):
    special_characters_pattern = re.compile(r"[#%@\<>{}()=\[\]*\-\\|_½]")
    return special_characters_pattern.sub("", text)


def preprocess_text(text):
    text = expand_contractions(text)
    text = remove_special_characters(text)

    tokens = nltk.word_tokenize(text)

    lemmatizer = nltk.stem.WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]
    lemmatized_tokens = [token.lower() for token in lemmatized_tokens]

    stop_words = set(nltk.corpus.stopwords.words('english'))
    filtered_tokens = [token for token in lemmatized_tokens if token not in stop_words]

    return filtered_tokens

df['text'] = df['text'].apply(preprocess_text)

<ipython-input-13-f1f03507685a>:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df['text'].apply(preprocess_text)


In [14]:
df.head()

,text,label
0,"[pentagon, upgrading, gps, sniper, atp, bahrai...",AVI
1,"[search, continues, japanese, air, selfdefense...",MIS
2,"[recent, photo, hill, afb, f35s, deployed, kad...",COC
3,"[kfx, :, south, korea, wants, build, fighter, ...",MIS
4,"[army, aviation, missile, command, awarded, si...",AVI


In [15]:
import time
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [16]:
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.1, random_state=42)

X_train = [' '.join(tokens) for tokens in X_train]
X_test = [' '.join(tokens) for tokens in X_test]

tfidf_vectorizer = TfidfVectorizer(max_features=20000)
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

In [17]:
from sklearn.naive_bayes import MultinomialNB

nb_model = MultinomialNB()

start = time.time()
nb_model.fit(X_train_tfidf, y_train)
y_pred_nb = nb_model.predict(X_test_tfidf)
end = time.time()

report = classification_report(y_test, y_pred_nb)

print(f"MultinomialNB Execution Time: {end-start}")
print("MultinomialNB Accuracy:", accuracy_score(y_test, y_pred_nb))
# print(f"MultinomialNB Macro Precision: {report['macro avg']['precision']}")
# print(f"MultinomialNB Macro Recall: {report['macro avg']['recall']}")
print(report)

MultinomialNB Execution Time: 0.16619420051574707
MultinomialNB Accuracy: 0.4954480114997604
              precision    recall  f1-score   support

         AVI       0.46      0.87      0.60       756
         COC       0.65      0.07      0.12       199
         ECS       0.00      0.00      0.00         5
         ELC       0.00      0.00      0.00         3
         FFC       0.00      0.00      0.00        14
         FLC       0.53      0.20      0.29        46
         FLU       0.00      0.00      0.00         2
         LAG       0.00      0.00      0.00        11
         MEC       0.00      0.00      0.00        11
         MIS       0.47      0.36      0.41       426
         OTH       0.72      0.43      0.54       428
         PRP       1.00      0.03      0.06        33
         WEA       0.71      0.12      0.20       142
         WIN       0.00      0.00      0.00        11

    accuracy                           0.50      2087
   macro avg       0.32      0.15      0.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [18]:
from sklearn.linear_model import LogisticRegression

lr_model = LogisticRegression(max_iter=20000)

start = time.time()
lr_model.fit(X_train_tfidf, y_train)
y_pred_lr = lr_model.predict(X_test_tfidf)
end = time.time()

report = classification_report(y_test, y_pred_lr)

print(f"Logistic Regression Execution Time: {end-start}")
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_lr))
# print(f"Logistic Regression Macro Precision: {report['macro avg']['precision']}")
# print(f"Logistic Regression Macro Recall: {report['macro avg']['recall']}")
print(report)

Logistic Regression Execution Time: 71.61025476455688
Logistic Regression Accuracy: 0.5816962146621946
              precision    recall  f1-score   support

         AVI       0.55      0.76      0.64       756
         COC       0.63      0.24      0.34       199
         ECS       0.00      0.00      0.00         5
         ELC       0.00      0.00      0.00         3
         FFC       1.00      0.07      0.13        14
         FLC       0.71      0.37      0.49        46
         FLU       0.00      0.00      0.00         2
         LAG       0.75      0.27      0.40        11
         MEC       0.00      0.00      0.00        11
         MIS       0.55      0.47      0.51       426
         OTH       0.66      0.65      0.65       428
         PRP       0.87      0.39      0.54        33
         WEA       0.64      0.54      0.58       142
         WIN       0.00      0.00      0.00        11

    accuracy                           0.58      2087
   macro avg       0.45      0.

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [19]:
from sklearn.svm import LinearSVC
svm_model = LinearSVC()

start = time.time()
svm_model.fit(X_train_tfidf, y_train)
y_pred_svm = svm_model.predict(X_test_tfidf)
end = time.time()

report = classification_report(y_test, y_pred_svm)

print(f"Linear SVM Execution Time: {end-start}")
print("Linear SVM Accuracy:", accuracy_score(y_test, y_pred_svm))
# print(f"Linear SVM Macro Precision: {report['macro avg']['precision']}")
# print(f"Linear SVM Macro Recall: {report['macro avg']['recall']}")
print(report)

Linear SVM Execution Time: 9.486808061599731
Linear SVM Accuracy: 0.5654048873981792
              precision    recall  f1-score   support

         AVI       0.56      0.67      0.61       756
         COC       0.49      0.27      0.35       199
         ECS       0.00      0.00      0.00         5
         ELC       0.00      0.00      0.00         3
         FFC       0.43      0.21      0.29        14
         FLC       0.49      0.41      0.45        46
         FLU       0.00      0.00      0.00         2
         LAG       0.36      0.36      0.36        11
         MEC       0.00      0.00      0.00        11
         MIS       0.54      0.49      0.51       426
         OTH       0.62      0.64      0.63       428
         PRP       0.71      0.52      0.60        33
         WEA       0.61      0.63      0.62       142
         WIN       0.75      0.27      0.40        11

    accuracy                           0.57      2087
   macro avg       0.40      0.32      0.34      

In [25]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

start = time.time()
rf_model.fit(X_train_tfidf, y_train)
y_pred_rf = rf_model.predict(X_test_tfidf)
end = time.time()

report = classification_report(y_test, y_pred_rf)

print(f"Random Forest Execution Time: {end-start}")
print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
# print(f"Random Forest Macro Precision: {report['macro avg']['precision']}")
# print(f"Random Forest Macro Recall: {report['macro avg']['recall']}")
print(report)

Random Forest Execution Time: 541.7447595596313
Random Forest Accuracy: 0.5342597029228557
              precision    recall  f1-score   support

         AVI       0.49      0.81      0.61       756
         COC       0.49      0.13      0.21       199
         ECS       0.00      0.00      0.00         5
         ELC       0.00      0.00      0.00         3
         FFC       0.50      0.29      0.36        14
         FLC       0.42      0.35      0.38        46
         FLU       0.00      0.00      0.00         2
         LAG       0.17      0.09      0.12        11
         MEC       0.00      0.00      0.00        11
         MIS       0.59      0.36      0.45       426
         OTH       0.67      0.54      0.60       428
         PRP       0.61      0.33      0.43        33
         WEA       0.67      0.44      0.53       142
         WIN       0.67      0.18      0.29        11

    accuracy                           0.53      2087
   macro avg       0.38      0.25      0.28